# Car Price prediction


## Прогнозирование стоимости автомобиля по характеристикам


* Построим "наивную"/baseline модель, предсказывающую цену по модели и году выпуска (с ней будем сравнивать другие модели)
* Обработаем и отнормируем признаки
* Сделаем первую модель на основе градиентного бустинга с помощью CatBoost
* Сделаем вторую модель на основе нейронных сетей и сравним результаты
* Сделаем multi-input нейронную сеть для анализа табличных данных и текста одновременно
* Добавим в multi-input сеть обработку изображений
* Осуществим ансамблирование градиентного бустинга и нейронной сети (усреднение их предсказаний)

In [1]:
!pip install -q tensorflow==2.3

In [2]:
#аугментации изображений
!pip install albumentations -q

In [3]:
!pip install pymystem3

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [7]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [8]:
!pip freeze > requirements.txt

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [9]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [10]:
train.info()

In [11]:
train.nunique()

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [12]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [13]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

# EDA

In [14]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

In [15]:
import pandas_profiling
pandas_profiling.ProfileReport(train)

### POTENTIAL NEXT STEPS IN VARIABLES CLEANING BASED ON THE PANDAS PROFILING:
- check whether *bodyType, model_info* values are unique
- check *enginePower, engineDisplacement* - can be transformed into numeric?
- check *mileage, modelDate, price, productionDate* outliers
- taking out *color, Руль* variable as it is not important / not available?
- check any additional info can be extracted from the *name, vehicleConfiguration, Владение*

### DUPLICATES:
no duplicated rows

### MISSING VALUES:
- Владельцы <0.1%
- Владение 64.7%

### OUTLIERS:
fields with potential outliers:
- mileage
- productionDate
- modelDate
- price


### STRONG CORRELATIONS:
- mileage ~ modelDate ~ productionDate >> to take out modelDate?

### VARIABLES:
**NUMERICAL FEATURES:**

 ['mileage', 'modelDate', 'productionDate']

**CATEGORICAL FEATURES:**

 ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
  
**HIGH CARDINALITY FEATURES:**
description, vehicleConfiguration, Владение

### bodyType

In [16]:
#bodyType values are unique, doesn't seem to make sense to standardize:
train.bodyType.value_counts()

### model_info

In [17]:
# model_info variable seems to be standardized as well, 
# the only question is whether "KLASSE" and "CLASSE" for Mercedes brand should be standardized as well:

train.model_info.unique()

In [18]:
train[train.brand == "MERCEDES"].model_info.value_counts()

### enginePower

In [19]:
# enginePower seems to be possible to convert to numeric value by taking out "N12"
train.enginePower.unique()

In [20]:
train['enginePower'] = train['enginePower'].astype(str).apply(lambda x: x.replace(" N12", ""))
test['enginePower'] = test['enginePower'].astype(str).apply(lambda x: x.replace(" N12", ""))
train['enginePower'] = train['enginePower'].astype(int)
test['enginePower'] = test['enginePower'].astype(int)

### engineDisplacement

In [21]:
# it is possible to convert engineDisplacement to a numeric value as well:
train.engineDisplacement.unique()

In [22]:
train['engineDisplacement'] = train['engineDisplacement'].astype(str).apply(lambda x: None if x == "undefined LTR" else x.replace(" LTR", ""))
test['engineDisplacement'] = test['engineDisplacement'].astype(str).apply(lambda x: None if x == "undefined LTR" else x.replace(" LTR", ""))
train['engineDisplacement'] = train['engineDisplacement'].astype(float)
test['engineDisplacement'] = test['engineDisplacement'].astype(float)

### Mileage

In [23]:
# checking outliers for mileage variable:
train.mileage.hist()

In [24]:
plt.figure(figsize=(10, 5))
plt.scatter(np.log(train.price), train.mileage)

In [25]:
train.mileage.max()

In [26]:
# removing the clear outlier:
train.drop(train[train.mileage > 999998].index, inplace=True)

### Price

In [27]:
# checking outliers for mileage variable:
train.price.hist()

### modelDate

In [28]:
# checking outliers for mileage variable:
train.modelDate.hist()

In [29]:
plt.figure(figsize=(10, 5))
plt.scatter(np.log(train.price), train.modelDate)

In [30]:
# normalizing modelDate with logarithm
train['modelDate'] = train.modelDate.apply(lambda x: np.log(x))
test['modelDate'] = test.modelDate.apply(lambda x: np.log(x))

### productionDate

In [31]:
# checking outliers for mileage variable:
train.productionDate.hist()

In [32]:
plt.figure(figsize=(10, 5))
plt.scatter(np.log(train.price), train.productionDate)

In [33]:
# normalizing productionDate with logarithm
train['modelDate'] = train.modelDate.apply(lambda x: np.log(x))
test['modelDate'] = test.modelDate.apply(lambda x: np.log(x))

### Руль

In [34]:
# in the train dataset there are just 2 cases of the right wheel car, so we can remove this variable from the set:
train.Руль.value_counts()

### Color

In [35]:
# color seem to be able to deliver some information
train.color.unique()

### Name

In [36]:
# checking whether we can extract any info from the name parameter
# name seems to be the composition of vehicleConfiguration, enginePower, and number of Doors
train.iloc[1044]

In [37]:
# removing info which is available in other columns:
train['name'] = train.apply(lambda row : row['name'].replace(str(row['enginePower']), ''), axis=1)
train['name'] = train.apply(lambda row : row['name'].replace(str(row['engineDisplacement']), ''), axis=1)

test['name'] = test.apply(lambda row : row['name'].replace(str(row['enginePower']), ''), axis=1)
test['name'] = test.apply(lambda row : row['name'].replace(str(row['engineDisplacement']), ''), axis=1)

In [38]:
# looks like the names will need to be cleaned based on the brand, since there are some overlaps
# pd.set_option("display.max_rows", None)
train.name.value_counts().sort_index(ascending=True).head(20)

In [39]:
# MERCEDES features from name:
train['name_AMG'] = train['name'].apply(lambda x: 1 if 'AMG' in x else 0)
test['name_AMG'] = test['name'].apply(lambda x: 1 if 'AMG' in x else 0)

train['name_L1'] = train['name'].apply(lambda x: 1 if 'L1' in x else 0)
test['name_L1'] = test['name'].apply(lambda x: 1 if 'L1' in x else 0)

train['name_L2'] = train['name'].apply(lambda x: 1 if 'L2' in x else 0)
test['name_L2'] = test['name'].apply(lambda x: 1 if 'L2' in x else 0)

train['name_L3'] = train['name'].apply(lambda x: 1 if 'L3' in x else 0)
test['name_L3'] = test['name'].apply(lambda x: 1 if 'L3' in x else 0)

train['name_CDI'] = train['name'].apply(lambda x: 1 if 'CDI' in x else 0)
test['name_CDI'] = test['name'].apply(lambda x: 1 if 'CDI' in x else 0)

train['name_Blue'] = train['name'].apply(lambda x: 1 if 'Blue' in x else 0)
test['name_Blue'] = test['name'].apply(lambda x: 1 if 'Blue' in x else 0)

train['name_hyb'] = train['name'].apply(lambda x: 1 if 'hyb' in x else 0)
test['name_hyb'] = test['name'].apply(lambda x: 1 if 'hyb' in x else 0)

train['name_hyb'] = train['name'].apply(lambda x: 1 if 'HYBRID' in x else 0)
test['name_hyb'] = test['name'].apply(lambda x: 1 if 'HYBRID' in x else 0)

# BMW features from name:
train['name_i'] = train['name'].apply(lambda x: 1 if 'i' in x else 0)
test['name_i'] = test['name'].apply(lambda x: 1 if 'i' in x else 0)

train['name_xDrive'] = train['name'].apply(lambda x: 1 if 'xDrive' in x else 0)
test['name_xDrive'] = test['name'].apply(lambda x: 1 if 'xDrive' in x else 0)


#AUDI features from name:
#N/A - not enough unique names to make sense

In [40]:
# cleaning out the name variable:

replace_list = [
    #all:
    "(л.с.)", '( л.с.)','AT', 'AMT', 'MT', '4WD', 'd', " ", "  "
    
    #mercedes:
    'L1', 'L2','L3', 'AMG', 'Long', 'длинный', 'BlueTEC', 'CVT', 'CDI', 'BlueEFFICIENCY', 'экстра', 'компактный', 'hyb', 'HYBRID', 'BlueEfficiency', 'Eition','5-sp', '4x4','5G-TRONIC','5G-Tronic','9G-TRONIC','Kompressor','4MIC', 
    
    #bmw:
    'xDrive', 'L', 'x', 'sDrive', 'is', 'si', "S ",  "Package", "Pack", "age","(136кВт)", "(126кВт)",
    
    "e"
]

def remove_multiple_strings(cur_string, replace_list):
  for cur_word in replace_list:
    cur_string = cur_string.replace(cur_word, '')
  return cur_string


train['name'] = train.apply(lambda row : remove_multiple_strings(row['name'], replace_list), axis=1)
test['name'] = test.apply(lambda row : remove_multiple_strings(row['name'], replace_list), axis=1)

In [41]:
test.name.value_counts().head(20)

### vehicleConfiguration

In [42]:
# vehicleConfiguration is a combinaiton of other fiels, so can be omitted
train[['vehicleConfiguration', 'bodyType', 'engineDisplacement', 'vehicleTransmission', 'enginePower']].head(20)

### Владение

In [43]:
# looks like Владельцы can deliver better info for the model and doesn't have missing values
train[['Владение','Владельцы']].head(20)

### Description

In [44]:
## adding extra tags from description (did not prove to be a good practise, and will be conducted in the next steps with NLP anyway)

#train['новый'] = train.description.apply(lambda x: 1 if 'новый' in x else 0)
#train['идеальн'] = train.description.apply(lambda x: 1 if 'идеальн' in x else 0)
#train['обмен'] = train.description.apply(lambda x: 1 if 'обмен' in x else 0)
#train['торг'] = train.description.apply(lambda x: 1 if 'торг' in x else 0)
#train['дилер'] = train.description.apply(lambda x: 1 if 'дилер' in x else 0)
#train['срочн'] = train.description.apply(lambda x: 1 if 'срочн' in x else 0)
#train['ухож'] = train.description.apply(lambda x: 1 if 'ухож' in x else 0)
#train['кож'] = train.description.apply(lambda x: 1 if 'кож' in x else 0)
#train['рестайл'] = train.description.apply(lambda x: 1 if 'рестайл' in x else 0)
#train['ремонт'] = train.description.apply(lambda x: 1 if 'ремонт' in x else 0)
#train['проблемы'] = train.description.apply(lambda x: 1 if 'проблемы' in x else 0)

#test['новый'] = test.description.apply(lambda x: 1 if 'новый' in x else 0)
#test['идеальн'] = test.description.apply(lambda x: 1 if 'идеальн' in x else 0)
#test['обмен'] = test.description.apply(lambda x: 1 if 'обмен' in x else 0)
#test['торг'] = test.description.apply(lambda x: 1 if 'торг' in x else 0)
#test['дилер'] = test.description.apply(lambda x: 1 if 'дилер' in x else 0)
#test['срочн'] = test.description.apply(lambda x: 1 if 'срочн' in x else 0)
#test['ухож'] = test.description.apply(lambda x: 1 if 'ухож' in x else 0)
#test['кож'] = test.description.apply(lambda x: 1 if 'кож' in x else 0)
#test['рестайл'] = test.description.apply(lambda x: 1 if 'рестайл' in x else 0)
#test['ремонт'] = test.description.apply(lambda x: 1 if 'ремонт' in x else 0)
#test['проблемы'] = test.description.apply(lambda x: 1 if 'проблемы' in x else 0)

## Correlations

In [45]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax = sns.heatmap(train.corr(),fmt='.1g',
                 annot=True, cmap='coolwarm')

### RESULTS:

high correlations between:
- engineDisplacement ~ enginePower >> keeping enginePower
- modelDate ~ productionDate ~ mileage >> keeping mileage

no correlation with price:
- taking out name_CDI
- taking out name_hyb

### Adding new features

In [46]:
train['modelAge'] = 2021 - train['modelDate']
train['mile_per_year'] = train['mileage'] / train['modelAge']

test['modelAge'] = 2021 - test['modelDate']
test['mile_per_year'] = test['mileage'] / test['modelAge']

### EDA RESULTS:

- no duplicates found
- missing values in *Владельцы* (<0.1%), *Владение* (64.7%) >> removed *Владение*
- outliers in *mileage*, *modelDate*, *productionDate*, *price* >> will be treated by the Scaler function
- potential features to be added: *age of the model*, *mileage per year*
- not relevant variables: *Владение*, *Руль*


### DATA PREPROCESSING RESULTS:

- *model_info* >> reduced cardinality by keeping the model number only and creating additional features from it:
     - name_AMG
     - name_CDI
     - name_Blue
     - name_xDrive
     - name_i
     - name_hyb
     
- *enginePower, engineDisplacement* >> transformed to numeric
- *modelDate*, *productionDate* >> transformed to logarithmic value
- removed *Владение*, *Руль*, as are not very informative
- removed *vehicleConfiguration*, as it is a combination of other features
- removed outliers in the *mileage* variable 
- due to high correlation, removed *modelDate*
- due to high correlation, removed *engineDisplacement*
- due to almost no correlation with price, removed *name_CDI*, *name_hyb*





In [47]:
# removing all unused columns
train.drop(['Владение', 'Руль', 'vehicleConfiguration', 'modelDate', 'engineDisplacement',
            #additionally removing the newly added features from the name, as they did not improve the results:
            'name_AMG','name_L1', 'name_L2', 'name_L3','name_Blue','name_i', 'name_xDrive','name_CDI',  'name_hyb'
           ], axis=1)
test.drop(['Владение', 'Руль', 'vehicleConfiguration', 'modelDate', 'engineDisplacement',
            #additionally removing the newly added features from the name, as they did not improve the results:
            'name_AMG','name_L1', 'name_L2', 'name_L3','name_Blue','name_i', 'name_xDrive','name_CDI',  'name_hyb'
           ], axis=1)

In [48]:
train.iloc[0]

# PreProc Tabular Data

### CHOOSING THE FEATURES TO TAKE IN:

In [49]:
# taking out several features identified in the EDA as irrelevant (i.e. vehicleConfiguration) or those that are now numerical (i.e. enginePower)
categorical_features = ['bodyType', 'brand', 'color', 
                        #'engineDisplacement', 'enginePower', 'vehicleConfiguration'
                        'fuelType', 'model_info', 'name',
                        'numberOfDoors', 'vehicleTransmission', 
                        'Владельцы', 'ПТС', 'Привод', 
                        #'Владение', 'Руль'
                       ]

# taking out engineDispacement due to high correlation with enginePower, adding new calculated features
numerical_features = ['mileage', 
                      'modelDate', 
                      'productionDate', 
                      #'engineDisplacement', 
                      'enginePower', 
                     'modelAge', 
                     'mile_per_year']

#additionaly added parameters from *name* column did not improve the results 
#take_out = [#'name_AMG','name_L1', 'name_L2', 'name_L3','name_Blue','name_i', 'name_xDrive'
           #'name_CDI',  'name_hyb']

In [50]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

In [51]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    
 
    df_output.drop(['description','sell_id','engineDisplacement', 'vehicleConfiguration',
                    'Владение', 'Руль', 'name_CDI',  'name_hyb',
                    'name_AMG','name_L1', 'name_L2', 'name_L3','name_Blue','name_i', 'name_xDrive',
                   ], axis = 1, inplace=True)
 
    
    # ################### Numerical Features ############################################################## 
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)
        
    
    # Нормализация данных
    #scaler = MinMaxScaler()
    #for column in numerical_features:
    #    df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    scaler = RobustScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        
    
    # ################### Categorical Features ############################################################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    

    
    return df_output

In [52]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [53]:
#df_preproc.iloc[0]

In [54]:
df_preproc.info()

## Split data

In [55]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values    
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [56]:
test_data.info()

# Model 2: CatBoostRegressor

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [58]:
model = CatBoostRegressor(iterations = 5000,
                          #depth=10,
                          #learning_rate = 0.5,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=100,
         use_best_model=True,
         #plot=True
         )

In [59]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

### CATBOOST RESULTS:
- 18.67%, 10K epochs, data preprocessing
- 13.34%, when returning back productionDate
- 13.23%, 50K epochs
- 13.43%, 50K epochs, dropped 'name' variable completely
- 13.08%, 50K epochs, removed all added categories from name
- 13.14%, 50K epochs, removed numberOfDoors
- 13.14%, 50K epochs, added additional tags based on the 'description' feature, which doesn't seem to add any value
- 13.09%, 50K epochs, changed to RobustScaler()
- 11.62%, 50K epochs, taking the logarithm of the y
- 11.09%, 5K epochs, adding new features 'modelAge', 'mile_per_year'



### Submission

In [60]:
sub_predict_catboost = np.exp(model.predict(X_sub))
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

# Model 3: Tabular NN

Построим обычную сеть:

In [61]:
X_train.head(5)

In [62]:
pd.set_option("display.max_rows", None)
X_train.iloc[0]

## Simple Dense NN

In [63]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [64]:
model.summary()

In [65]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [66]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor='val_MAPE', save_best_only=True, verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

### Fit

In [67]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [68]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [69]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

### Simple Dense NN RESULTS:
- 10.60% after the initial model run
- 11.15% changed activation in the first Dense layer from relu to sigmoid
- 10.6% reverted back to relu, added an additional Dense layer

Model works 1% better than Catboost.
Overfitting is still an issue, but it is rather small.


In [70]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [71]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

Рекомендации для улучшения Model 3:    
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример:
`modelDateNorm = np.log(2020 - data['modelDate'])`
Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста:
Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков
Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'), поэтому эти данные можно удалить. Затем следует еще сильнее сократить размерность, например, выделив наличие xDrive в качестве отдельного признака.

# Model 4: NLP + Multiple Inputs

In [72]:
data.description.head(2)

In [73]:
import nltk
from nltk.stem import *
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
import string
from tqdm.auto import tqdm, trange
from nltk import word_tokenize
nltk.download('punkt')

In [74]:
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

In [75]:
data['text_prep'] = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(data['description'])]

In [76]:
data['text_prep'].head(5)

In [77]:
# loading the most common words for the russian language, so that we can remove them
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д', '●', '∙', '""', "''", "“", "———————————————————————————", "•", "″"])

In [78]:
# cleaning the text_prep from the most common stop words
text_prep_cleaned = []
for text in tqdm(data['text_prep']):
    tokens = word_tokenize(text)    
    cleaned_tokens = [token for token in tokens if token not in russian_stopwords]
    text = " ".join(cleaned_tokens)
    text_prep_cleaned.append(text)
    
data['text_prep_cleaned'] = text_prep_cleaned

In [79]:
pd.set_option('display.max_colwidth', None)
data['text_prep_cleaned'].head(2)

In [80]:
from pymystem3 import Mystem
mystem = Mystem()

In [81]:
# lemmatizing the text
text_prep_lem = []
for text in tqdm(data['text_prep']):
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        text_prep_lem.append(text)
    except Exception as e:
        print(e)
    
data['text_prep_lem'] = text_prep_lem

In [82]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [83]:
# split данных
text_train = data.text_prep_lem.iloc[X_train.index]
text_test = data.text_prep_lem.iloc[X_test.index]
text_sub = data.text_prep_lem.iloc[X_sub.index]

### Tokenizer

In [84]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.text_prep_lem)

In [85]:
#tokenize.word_index

In [86]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [87]:
# вот так теперь выглядит наш текст
print(text_train.iloc[7])
print(text_train_sequences[7])

### RNN NLP

In [88]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

### MLP

In [89]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

### Multiple Inputs NN

In [90]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [91]:
model.summary()

### Fit

In [92]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [93]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor='val_MAPE',save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=30, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [94]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [95]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [96]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [97]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

### NLP RESULTS:
- 11.10% - initial run, text tokenization
- 10.62% - lower-cased, removed punctuation, numbers and multiple spaces
- 10.71% - additionally removed russion stop-words
- 10.69% - additionally lemmatized the cleaned output
- 10.64% - returned back to text_prep column, where the stop words were not removed and lemmatized it
- 10.60% - changed earlystop patience to = 30, after epoch 100 we see almost no overfitting, but the graph is also very steep, so probably does not make much sense to run over higher number of epochs


NOTE:
- removing stop words did not significantly improve the model
- lemmatizing the text did not significantly improve the model




In [98]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission.csv', index=False)

# Model 5: Добавляем картинки

### Data

In [99]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [100]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

### albumentations

In [101]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, CenterCrop
)


# applying augmentations on the images
augmentation = Compose([
    HorizontalFlip(p=0.5),
    #OneOf([
    #    IAAAdditiveGaussianNoise(),
    #    GaussNoise(),
    #], p=0.1),
    #OneOf([
    #    MotionBlur(p=0.2),
    #    MedianBlur(blur_limit=3, p=0.1),
    #    Blur(blur_limit=3, p=0.1),
    #], p=0.2),
    #ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=1),
    #OneOf([
    #    OpticalDistortion(p=0.3),
    #    GridDistortion(p=0.1),
    #    IAAPiecewiseAffine(p=0.3),
    #], p=0.2),
    #OneOf([
    #    CLAHE(clip_limit=2),
    #    IAASharpen(),
    #    IAAEmboss(),
    #    RandomBrightnessContrast(),
    #], p=0.3),
    #HueSaturationValue(p=0.3),
    #OneOf([
    #    CenterCrop(height=240, width=220),
    #    CenterCrop(height=220, width=240),
    #], p=0.2),
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [102]:
def make_augmentations(images):
  print('применение аугментаций', end = '')
  augmented_images = np.empty(images.shape)
  for i in range(images.shape[0]):
    if i % 200 == 0:
      print('.', end = '')
    augment_dict = augmentation(image = images[i])
    augmented_image = augment_dict['image']
    augmented_images[i] = augmented_image
  print('')
  return augmented_images

## tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод .fit() модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является [tf.data.Dataset](https://www.tensorflow.org/guide/data). Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с tf.data.Dataset.

Dataset хранит информацию о конечном или бесконечном наборе кортежей (tuple) с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары (input, target) для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости ([lazy evaluation](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BD%D0%B8%D0%B2%D1%8B%D0%B5_%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)).

`tf.data.Dataset.from_tensor_slices(data)` - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если `data = (np.zeros((128, 256, 256)), np.zeros(128))`, то датасет будет содержать 128 элементов, каждый из которых содержит один массив 256x256 и одно число.

`dataset2 = dataset1.map(func)` - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в датасете 1 и возвращать столько, сколько нужно иметь в датасете 2. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: `dataset2 = dataset.map(lambda img, label: img)`.

`dataset2 = dataset1.batch(8)` - группировка по батчам; если датасет 2 должен вернуть один элемент, то он берет из датасета 1 восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

`dataset.__iter__()` - превращение датасета в итератор, из которого можно получать элементы методом `.__next__()`. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом for.

`dataset2 = dataset1.repeat(X)` - датасет 2 будет повторять датасет 1 X раз.

Если нам нужно взять из датасета 1000 элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

`test_dataset = dataset.take(1000)
train_dataset = dataset.skip(1000)`

Датасет по сути неизменен: такие операции, как map, batch, repeat, take, skip никак не затрагивают оригинальный датасет. Если датасет хранит элементы [1, 2, 3], то выполнив 3 раза подряд функцию dataset.take(1) мы получим 3 новых датасета, каждый из которых вернет число 1. Если же мы выполним функцию dataset.skip(1), мы получим датасет, возвращающий числа [2, 3], но исходный датасет все равно будет возвращать [1, 2, 3] каждый раз, когда мы его перебираем.

tf.Dataset всегда выполняется в graph-режиме (в противоположность eager-режиму), поэтому либо преобразования (`.map()`) должны содержать только tensorflow-функции, либо мы должны использовать tf.py_function в качестве обертки для функций, вызываемых в `.map()`. Подробнее можно прочитать [здесь](https://www.tensorflow.org/guide/data#applying_arbitrary_python_logic).

In [103]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
#tokenize.fit_on_texts(data.description)
tokenize.fit_on_texts(data.text_prep_lem)

In [104]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(text_prep_lem):
  return sequence.pad_sequences(tokenize.texts_to_sequences(text_prep_lem), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.text_prep_lem.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.text_prep_lem.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.text_prep_lem.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

### Строим сверточную сеть для анализа изображений без "головы"

In [105]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

### Model finetuning

In [106]:
# first train only top layers (which were randomly initialized)
#efficientnet_model.trainable = True

# finetune from this layer onwards:
#fine_tune_at = len(efficientnet_model.layers)//2

# freeze all the layers before the fine_tune_at_layer:
#for layer in efficientnet_model.layers[:fine_tune_at]:
#    layer.trainable=False
        
# checking the trainable status of the individual layers of the model:
#for layer in efficientnet_model.layers:
#    print(layer, layer.trainable)    

In [107]:
len(model.trainable_variables)

In [108]:
#строим нейросеть для анализа табличных данных
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    ])

In [109]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [110]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
model.summary()

In [111]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [112]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor='val_MAPE',save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=15, restore_best_weights=True,)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=1e-5, verbose=1)
callbacks_list = [checkpoint, earlystop]

In [113]:
history = model.fit(train_dataset.batch(30),
                    epochs=300,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [114]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [115]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [116]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

### NN MODEL RESULTS:

- 10.92% initial model with adjusted augmentations
- 10.93%, but the model graph is showing that there is some overfitting going on
    - reduced number of augmenations
    - updated tabular model (added a new Dense layer)
    - introduced finetuning at the half of the layers of Efficientnet base model
    - added DropOut layer in the NLP model
    - changed earlystop patience to 15
- 10.85%, same but removed the finetuning
    
    


In [117]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)


#### Общие рекомендации:
* Попробовать разные архитектуры
* Провести более детальный анализ результатов
* Попробовать различные подходы в управление LR и оптимизаторы
* Поработать с таргетом
* Использовать Fine-tuning

#### Tabular
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример:
`modelDateNorm = np.log(2020 - data['modelDate'])`
Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста:
Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков
Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Можно удалить эти данные. Затем можно еще сильнее сократить размерность, например выделив наличие xDrive в качестве отдельного признака.

* Поработать над Feature engineering



#### NLP
* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа
Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста



#### CV
* Попробовать различные аугментации
* Fine-tuning

# Blend

In [118]:
blend_predict = (test_predict_catboost + test_predict_nn3[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [119]:
blend_sub_predict = (sub_predict_catboost + sub_predict_nn3[:,0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

# Model Bonus: проброс признака

In [120]:
# MLP
#model_mlp = Sequential()
#model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
#model_mlp.add(L.Dropout(0.5))
#model_mlp.add(L.Dense(256, activation="relu"))
#model_mlp.add(L.Dropout(0.5))

In [121]:
# FEATURE Input
# Iput
#productiondate = L.Input(shape=[1], name="productiondate")
# Embeddings layers
#emb_productiondate = L.Embedding(len(X.productionDate.unique().tolist())+1, 20)(productiondate)
#f_productiondate = L.Flatten()(emb_productiondate)

In [122]:
#combinedInput = L.concatenate([model_mlp.output, f_productiondate,])
# being our regression head
#head = L.Dense(64, activation="relu")(combinedInput)
#head = L.Dense(1, activation="linear")(head)

#model = Model(inputs=[model_mlp.input, productiondate], outputs=head)

In [123]:
#model.summary()

In [124]:
#optimizer = tf.keras.optimizers.Adam(0.01)
#model.compile(loss='MAPE',optimizer=optimizer, metrics='MAPE')

In [125]:
#history = model.fit([X_train, X_train.productionDate.values], y_train,
#                    batch_size=512,
#                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
#                    validation_data=([X_test, X_test.productionDate.values], y_test),
#                    callbacks=callbacks_list
#                   )

In [126]:
#model.load_weights('../working/best_model.hdf5')
#test_predict_nn_bonus = model.predict([X_test, X_test.productionDate.values])
#print(f"TEST mape: {(mape(y_test, test_predict_nn_bonus[:,0]))*100:0.2f}%")